# Notebook for the experiments
In this notebook are contained the following features:
* GRAFF + Link prediction,

The main tools that have been exploited are [PyTorch](https://pytorch.org/) (1.13.0), [PyTorch-Lightning](https://www.pytorchlightning.ai/index.html) (1.5.10), [Pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html) (2.3.0) and [Weights & Biases](https://wandb.ai/)

### Requirements to run the notebook

In [ ]:
# !pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
# !pip install pytorch-lightning==1.5.10
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
# !pip install torch_geometric
# !pip install wandb
# !pip install ogb

## Importing the libraries

In [ ]:
######## IMPORT EXTERNAL FILES ###########
import torch
import torch.nn.functional as F
import torch.nn.utils.parametrize as parametrize
import torch.nn as nn

import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import negative_sampling
from torch_geometric.nn import GATConv, GCNConv, SAGEConv

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.loggers import WandbLogger

import numpy as np
import wandb
######### IMPORT INTERNAL FILES ###########
import sys
sys.path.append("../../src")
from GRAFF import *
from config import *
from utils import set_seed

### System configuration

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_gpus = 1 if device == 'cuda' else 0
set_seed(SEED)


if wb:
    wandb.login()

## PyTorch Lightning DataModule (Link Prediction)

In [ ]:
class DataModuleLP(pl.LightningDataModule):

    def __init__(self,  train_set, val_set, test_set, mode, batch_size):

        self.mode = mode  # "hp" or "test"
        self.batch_size = batch_size
        self.train_set, self.val_set, self.test_set = train_set, val_set, test_set

    def setup(self, stage=None):
        if stage == 'fit':

            # edge_index are the message passing edges,
            # edge_label_index are the supervision edges.
            if self.train_set.pos_edge_label_index.shape[1] < self.train_set.edge_index.shape[1]:
                pos_mask_edge = self.train_set.pos_edge_label_index.shape[1]

                self.train_set.edge_index = self.train_set.edge_index[:,
                                                                      pos_mask_edge:]
            else:
                self.train_set.pos_edge_label_index = self.train_set.edge_index[:,
                                                                                :self.train_set.edge_index.shape[1] // 2]
                # self.train_set.neg_edge_label_index = self.train_set.neg_edge_label_index[
                #     :, :self.train_set.edge_index.shape[1] // 2]

                self.train_set.edge_index = self.train_set.edge_index[:,
                                                                      self.train_set.edge_index.shape[1] // 2:]

    def train_dataloader(self, *args, **kwargs):
        return DataLoader([self.train_set], batch_size=batch_size, shuffle=False)

    def val_dataloader(self, *args, **kwargs):
        if self.mode == 'hp':
            return DataLoader([self.val_set], batch_size=batch_size, shuffle=False)
        elif self.mode == 'test':
            return DataLoader([self.test_set], batch_size=batch_size, shuffle=False)
    
    def test_dataloader(self, *args, **kwargs):
        if self.mode == 'hp':
            return DataLoader([self.val_set], batch_size=batch_size, shuffle=False)
        elif self.mode == 'test':
            return DataLoader([self.test_set], batch_size=batch_size, shuffle=False)

In [ ]:
train_data = torch.load(dataset_name + "/train_data.pt")
val_data = torch.load(dataset_name + "/val_data.pt")
test_data = torch.load(dataset_name + "/test_data.pt")

In [ ]:
# print(train_data)
# print(val_data)
# print(test_data)

In [ ]:
mode = 'hp'  # hp: Hyperparameter selection mode
sweep = True
dataM = DataModuleLP(train_data.clone(), val_data.clone(), test_data.clone(), mode = mode, batch_size = batch_size)
dataM.setup(stage='fit')
dataM.setup(stage='test') 


In [ ]:
print(dataM.train_set)
print(dataM.val_set)
print(dataM.test_set)


### PyTorch Lightning Callbacks

In [ ]:

class Get_Metrics(Callback):

    def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"):

        # Compute the metrics
        train_loss = sum(
            pl_module.train_prop['loss']) / len(pl_module.train_prop['loss'])
        train_auroc = sum(
            pl_module.train_prop['AUROC'])/len(pl_module.train_prop['AUROC'])
        # train_acc100 = sum(
        #     pl_module.train_prop['HR@100']) / len(pl_module.train_prop['HR@100'])
        # train_acc20 = sum(
        #     pl_module.train_prop['HR@20']) / len(pl_module.train_prop['HR@20'])
        # train_acc1 = sum(
        #     pl_module.train_prop['HR@1']) / len(pl_module.train_prop['HR@1'])
        test_loss = sum(
            pl_module.test_prop['loss']) / len(pl_module.test_prop['loss'])
        test_auroc = sum(
            pl_module.test_prop['AUROC'])/len(pl_module.test_prop['AUROC'])
        # test_acc100 = sum(pl_module.test_prop['HR@100']) / \
        #     len(pl_module.test_prop['HR@100'])
        # test_acc20 = sum(pl_module.test_prop['HR@20']) / \
        #     len(pl_module.test_prop['HR@20'])
        # test_acc1 = sum(pl_module.test_prop['HR@1']) / \
        #     len(pl_module.test_prop['HR@1'])

        # Log the metrics
        pl_module.log(name='Loss on train', value=train_loss,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='Loss on test', value=test_loss,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='AUROC on train', value=train_auroc,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='AUROC on test', value=test_auroc,
                      on_epoch=True, prog_bar=True, logger=True)
        # pl_module.log(name='HR@100 on train', value=train_acc100,
        #               on_epoch=True, prog_bar=True, logger=True)
        # pl_module.log(name='HR@100 on test', value=test_acc100,
        #               on_epoch=True, prog_bar=True, logger=True)

        # pl_module.log(name='HR@20 on train', value=train_acc20,
        #               on_epoch=True, prog_bar=True, logger=True)
        # pl_module.log(name='HR@20 on test', value=test_acc20,
        #               on_epoch=True, prog_bar=True, logger=True)

        # pl_module.log(name='HR@1 on train', value=train_acc1,
        #               on_epoch=True, prog_bar=True, logger=True)
        # pl_module.log(name='HR@1 on test', value=test_acc1,
        #               on_epoch=True, prog_bar=True, logger=True)
        pl_module.last_metric = test_auroc

        # Re-initialize the metrics
        pl_module.train_prop['loss'] = []
        pl_module.train_prop['HR@100'] = []
        pl_module.train_prop['HR@20'] = []
        pl_module.train_prop['HR@1'] = []
        pl_module.train_prop['AUROC'] = []


        pl_module.test_prop['loss'] = []
        pl_module.test_prop['HR@100'] = []
        pl_module.test_prop['HR@20'] = []
        pl_module.test_prop['HR@1'] = []
        pl_module.test_prop['AUROC'] = []


## PyTorch Lightning Training Module (Node Classification)

In [ ]:
import torch
from sklearn import metrics


class TrainingModule(pl.LightningModule):

    def __init__(self, model, predictor, lr, wd):
        super().__init__()
        self.model = model.to(device)
        self.predictor = predictor.to(device)
        self.lr = lr
        self.wd = wd

        self.train_prop = {'loss': [], 'HR@100': [],
                           'HR@20': [], 'HR@1': [], 'AUROC': []}
        self.test_prop = {'loss': [], 'HR@100': [],
                          'HR@20': [], 'HR@1': [], 'AUROC': []}

    def training_step(self, batch, batch_idx):

        out = self.model(batch)

        pos_edge = batch.pos_edge_label_index

        pos_pred = self.predictor(
            out[pos_edge[0]], out[pos_edge[1]], training=True)

        neg_edge = batch.neg_edge_label_index

        neg_pred = self.predictor(
            out[neg_edge[0]], out[neg_edge[1]], training=True)

        loss = -torch.log(pos_pred + 1e-15).mean() - \
            torch.log(1 - neg_pred[:pos_pred.shape[0]] + 1e-15).mean()

        # acc100 = evaluate(
        #     pos_pred, neg_pred[:pos_pred.shape[0]], k = 100) #: 2*pos_pred.shape[0]+1000], k=100)
        # acc20 = evaluate(pos_pred, neg_pred, k = 20)
        # acc1 = evaluate(pos_pred, neg_pred, k = 1)
        auroc = compute_auroc(pos_pred, neg_pred[:pos_pred.shape[0]])

        self.train_prop['loss'].append(loss)
        self.train_prop['AUROC'].append(auroc)
        # self.train_prop['HR@100'].append(acc100)
        # self.train_prop['HR@20'].append(acc20)
        # self.train_prop['HR@1'].append(acc1)

        return loss

    def validation_step(self, batch, batch_idx):

        out = self.model(batch)

        pos_edge = batch.pos_edge_label_index

        # training is for dropout
        pos_pred = self.predictor(
            out[pos_edge[0]], out[pos_edge[1]], training=False)

        neg_edge = batch.neg_edge_label_index

        # training is for dropout
        neg_pred = self.predictor(
            out[neg_edge[0]], out[neg_edge[1]], training=False)

        loss = -torch.log(pos_pred + 1e-15).mean() - \
            torch.log(1 - neg_pred[:pos_pred.shape[0]] + 1e-15).mean()

        # acc100 = evaluate(
        #     pos_pred, neg_pred[pos_pred.shape[0]: 2*pos_pred.shape[0]+1000], k=100)
        # acc20 = evaluate(pos_pred, neg_pred, k = 20)
        # acc1 = evaluate(pos_pred, neg_pred, k = 1)
        auroc = compute_auroc(pos_pred, neg_pred[:pos_pred.shape[0]])

        self.test_prop['loss'].append(loss)
        self.test_prop['AUROC'].append(auroc)
        # self.test_prop['HR@100'].append(acc100)
        # self.test_prop['HR@20'].append(acc20)
        # self.test_prop['HR@1'].append(acc1)

        return loss

    def testing_step(self, batch, batch_idx, view=False):
        sets = {0: ['train', dataM.train_set], 1: [
            'validation', dataM.val_set], 2: ['test', dataM.test_set]}
        self.eval()
        with torch.no_grad():
            for idx in range(len(sets)):
                print("THESE ARE THE PREDICTION FOR THE {}: \n".format(
                    sets[idx][0]))
                data = sets[idx][1]
                out = self.model(data)

                preds = self.predictor(
                    out[data.pos_edge_label_index[0]], out[data.pos_edge_label_index[1]], training=False)
                neg_shape = preds.shape[0]
                preds_neg = self.predictor(out[data.neg_edge_label_index[0][:preds.shape[0]]],  # [neg_shape: neg_shape*2 + 1000]],
                                        out[data.neg_edge_label_index[1][:preds.shape[0]]], training=False)  # [neg_shape: neg_shape*2 + 1000]]

                # preds_neg = self.predictor(out[data.neg_edge_label_index[0][:neg_shape]],
                #                     out[data.neg_edge_label_index[1][:neg_shape]], training=False)
                auroc = compute_auroc(preds, preds_neg)
                print("AUROC is: ", auroc)

                print("The mean predictions on positives are: \n", torch.mean(preds))
                print("While the shape of the predicted positives is: \n", preds.shape)

                print("The mean predictions on negatives are: \n",
                    torch.mean(preds_neg))
                print("While the shape of the predicted negatives is: \n", preds_neg.shape)

                print("The HR@100 is: \n", evaluate(preds, preds_neg, k=100))

                if view:
                    print("POS: \n", preds)
                    print("NEG: \n", preds_neg)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            list(self.model.parameters()) + list(self.predictor.parameters()), lr=self.lr, weight_decay=self.wd)
        return self.optimizer


def compute_auroc(positives_tensor, negatives_tensor):
    # Combine the positive and negative tensors
    all_tensor = torch.cat([positives_tensor, negatives_tensor])

    # Create the corresponding labels (1 for positives, 0 for negatives)
    labels = torch.cat([torch.ones_like(positives_tensor), torch.zeros_like(negatives_tensor)])

    # Calculate the AUROC using sklearn's roc_auc_score function
    auroc = metrics.roc_auc_score(labels.detach().cpu().numpy(), all_tensor.detach().cpu().numpy())

    return auroc


# def compute_auroc(positives_tensor, negatives_tensor):
#     # Combine the positive and negative tensors
#     all_tensor = torch.cat([positives_tensor, negatives_tensor])

#     # Create the corresponding labels (1 for positives, 0 for negatives)
#     labels = torch.cat([torch.ones_like(positives_tensor),
#                        torch.zeros_like(negatives_tensor)])

#     # Calculate the AUROC using sklearn's roc_auc_score function

#     fpr, tpr, thresholds = metrics.roc_curve(y_true=labels.detach().cpu().numpy(
#     ), y_score=all_tensor.detach().cpu().numpy(), pos_label=1)  # positive class is 1; negative class is 0
#     auroc = metrics.auc(fpr, tpr)

#     return auroc


def evaluate(pos_pred, neg_pred, k=100):
    n_indices = pos_pred.shape[0]
    hr = 0
    # print(neg_pred.shape)
    k = min(neg_pred.shape[0]+1, k)

    for pos_idx in range(n_indices):
        pos = pos_pred[pos_idx].unsqueeze(0)
        eps = 0.1
        # Checking if the predictions are the same over all the negative distribution
        if round(torch.mean(neg_pred).item(), 4)-eps <= round(pos.item(), 4) <= round(torch.mean(neg_pred).item(), 4) + eps and \
            round(torch.min(neg_pred).item(), 4)-eps <= round(pos.item(), 4) <= round(torch.min(neg_pred).item(), 4) + eps and \
                round(torch.max(neg_pred).item(), 4)-eps <= round(pos.item(), 4) <= round(torch.max(neg_pred).item(), 4) + eps:
            continue
        tot_tensor = torch.cat((neg_pred, pos), dim=0)
        # print("tot_tensor: ", tot_tensor.shape)

        scores_idx = torch.topk(tot_tensor.squeeze(1), k).indices
        # print("SCORES: ", scores_idx)

        # Check if the positive is in the top100. Positive is marked by the neg_pred.shape[0]
        if neg_pred.shape[0] in scores_idx:
            # print("POS: ", round(pos.item(), 4))
            # print("MEAN: ", round(torch.mean(neg_pred).item(), 4))
            # print("MIN: ", round(torch.min(neg_pred).item(), 4))
            # print("MAX: ", round(torch.max(neg_pred).item(), 4))

            hr += 1
    return hr/n_indices

In [ ]:

# hp enables a grid search on a wide set of hyperparameters.
if not sweep or mode == 'test':
    model = PhysicsGNN_LP(dataset, hidden_dim, num_layers, input_dropout, step=step)
    # model = GNN_LP(dataset, hidden_dim, num_layers, GNN=GCNConv)
    # model = GNNStack(dataset.x.shape[1], hidden_dim, hidden_dim, num_layers, dropout, emb=True)
    predictor = LinkPredictor(
        hidden_dim, output_dim, mlp_layer, link_bias, dropout, device=device)
    # predictor = LinkPredictor(
    #      hidden_dim, hidden_dim, 1, num_layers,
    #              dropout)

    pl_training_module = TrainingModule(model, predictor, lr, wd)

### Hyperparameters Tuning

In [13]:
def sweep_train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
    
        metrics = []
        for run in range(runs):
            print("RUN n° {}".format(run+1))
            # model = PhysicsGNN_LP(dataset, hidden_dim,
            #                     num_layers, input_dropout, step=step)
            model = GNN_LP(dataset, hidden_dim, num_layers, GNN=GCNConv)
            predictor = LinkPredictor(
                hidden_dim, output_dim, config.mlp_layer, link_bias, dropout, device=device)
            # model = GNNStack(dataset.x.shape[1], config.hidden_dim, config.hidden_dim, config.num_layers, config.dropout, emb=True)

            # predictor = LinkPredictor(
            #     config.hidden_dim, config.hidden_dim, 1, config.num_layers+1,
            #             config.dropout)
            pl_training_module = TrainingModule(
                model, predictor, lr, wd)
            exp_name = "Sweep_LinkPred"
            wandb_logger = WandbLogger(
                project=project_name, name=exp_name, config=hyperparameters)
            trainer = trainer = pl.Trainer(
                max_epochs=epochs,  # maximum number of epochs.
                gpus=num_gpus,  # the number of gpus we have at our disposal.
                default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=10), EarlyStopping('AUROC on test', mode='max', patience=15)],
                logger=wandb_logger
            )
            trainer.fit(model=pl_training_module, datamodule=dataM)
            metrics.append(pl_training_module.last_metric)
        metrics = np.array(metrics)
        mean, std = np.mean(metrics), np.std(metrics)
        wandb.log({"AUROC on test (Mean)": mean, "AUROC on test (Std.)": std})


if mode == 'hp' and sweep:

    import pprint

    pprint.pprint(sweep_config)

    sweep_id = wandb.sweep(sweep_config, project=project_name)

    wandb.agent(sweep_id, sweep_train, count=15)

    wandb.finish()

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'AUROC on test (Mean)'},
 'parameters': {'mlp_layer': {'values': [0, 1, 2, 3, 4]}}}
Create sweep with ID: r7rmc4mx
Sweep URL: https://wandb.ai/deepl_wizards/Link%20Prediction%20with%20PBGNN/sweeps/r7rmc4mx


wandb: Agent Starting Run: vkll7mj5 with config:
wandb: 	mlp_layer: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | GNN_LP        | 501 K 
1 | predictor | LinkPredictor | 20.5 K
--------------------------------------------
522 K     Trainable params
0         Non-trainable params
522 K     Total params
2.089     Total estimated model params size (MB)
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/peppe/Desktop/Università/Projects/Thesis/Link_Predictio

RUN n° 1
                                                              

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 21022
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (1) is smaller than the logging inter

...

Epoch 1:  50%|█████     | 1/2 [00:00<00:00, 140.26it/s, loss=1.39, v_num=dry1, Loss on train=1.390, Loss on test=1.390, AUROC on train=0.468, AUROC on test=0.417]

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


Epoch 20: 100%|██████████| 2/2 [00:00<00:00, 84.77it/s, loss=0.895, v_num=dry1, Loss on train=0.148, Loss on test=4.790, AUROC on train=0.994, AUROC on test=0.737] 


/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


AUROC on test,▄▆▅▅▄█▆▅▅▆▅▅▄▆▅▅▁█▆▅▅▅▅▅▆▆▅▅▁▇▆▅▇▅▅▅▆▅▅▄
AUROC on test (Mean),▁
AUROC on test (Std.),▁
AUROC on train,▆████▆██▃▇██▆███▁▇██▃███████▁███▆███████
Loss on test,▁▁▁▂█▁▁▁▁▁▁▄▁▁▁▂▁▁▁▁▁▁▁▄▁▁▁▄▁▁▁▁▁▁▂▇▁▁▁█
Loss on train,██▇▃▁█▇▅██▇▅██▆▃██▇▄██▆▅██▄▂██▇▄██▅▄█▇▄▁
epoch,▁▃▄▇█▂▄▅▁▂▄▆▁▄▅▇▁▂▄▆▁▃▅▆▂▄▆▇▁▃▅▆▂▃▅▇▂▄▆█
trainer/global_step,▁▃▄▆█▂▄▅▁▂▅▆▁▃▅▇▁▂▅▆▂▃▅▇▂▄▆█▂▃▅▇▂▄▅▁▃▅▆█
AUROC on test,0.7373
AUROC on test (Mean),0.74492
AUROC on test (Std.),0.01039


In [ ]:
if wb:
    # exp_name = "Node_class_lr: " + \
    #     str(hyperparameters['learning rate']) + \
    #     '_wd: ' + str(hyperparameters['weight decay'])
    exp_name = ' '
    description = ' initial tests'
    exp_name += description
    wandb_logger = WandbLogger(
        project=project_name, name=exp_name, config=hyperparameters)


trainer = trainer = pl.Trainer(
    max_epochs=epochs,  # maximum number of epochs.
    gpus=num_gpus,  # the number of gpus we have at our disposal.
    default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=10), EarlyStopping('AUROC on test', mode='max', patience=15)],
    logger=wandb_logger if wb else None

)

In [ ]:
trainer.fit(model=pl_training_module, datamodule=dataM)
if wb:
    wandb.finish()

In [ ]:
pl_training_module.testing_step(None, None, view=False)

In [ ]:
data = dataM.val_set.to(device)
pl_training_module.to(device)
out = pl_training_module.model(data)

preds = pl_training_module.predictor(
    out[data.pos_edge_label_index[0]], out[data.pos_edge_label_index[1]], training=False)
neg_shape = preds.shape[0]
preds_neg = pl_training_module.predictor(out[data.neg_edge_label_index[0][neg_shape: neg_shape*2 + 1000]],
                                         out[data.neg_edge_label_index[1][neg_shape: neg_shape*2 + 1000]], training=False)

In [ ]:
print(torch.mean(preds_neg))
pos = torch.tensor([[torch.mean(preds).item()]], device=device)
print(pos)

In [ ]:
import sklearn.metrics

fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_true = true_labels, y_score = pred_probs, pos_label = 1) #positive class is 1; negative class is 0
auroc = sklearn.metrics.auc(fpr, tpr)

In [ ]:
hr = 0
for pos in preds:
   
    tot = torch.cat((preds_neg, pos.unsqueeze(1)), dim = 0)
    top = torch.topk(tot.squeeze(1), 100)
  

    if preds_neg.shape[0] in top.indices:
        hr+=1
        print(hr)

print(hr/preds.shape[0])